## ConvNets in TensorFlow!

In this part of this assignment, we will work with TensorFlow to write a convolutional neural network!

Credits to CS231N Assignment 2 (http://cs231n.github.io/assignments2018/assignment2/)

### 1. Look at this code and understand it!

This is a very simple 2-layer convnet. Understanding this code will help you with the next part.

In [4]:
import tensorflow as tf
import numpy as np

In [5]:
# clear old variables
tf.reset_default_graph()

# setup input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

def simple_model(X,y):
    # define our weights (e.g. init_two_layer_convnet)
    
    # setup variables
    Wconv1 = tf.get_variable("Wconv1", shape=[7, 7, 3, 32])
    bconv1 = tf.get_variable("bconv1", shape=[32])
    W1 = tf.get_variable("W1", shape=[5408, 10])
    b1 = tf.get_variable("b1", shape=[10])

    # define our graph (e.g. two_layer_convnet)
    a1 = tf.nn.conv2d(X, Wconv1, strides=[1,2,2,1], padding='VALID') + bconv1
    h1 = tf.nn.relu(a1)
    h1_flat = tf.reshape(h1,[-1,5408])
    y_out = tf.matmul(h1_flat,W1) + b1
    return y_out

## 2. Train a specific model!

Using the code provided above as guidance, and using TensorFlow documentation, specify a model with the following architecture:

1. 7x7 Convolutional Layer with 32 filters and stride of 1

2. ReLU Activation Layer

3. Spatial Batch Normalization Layer (trainable parameters, with scale and centering)

4. 2x2 Max Pooling layer with a stride of 2

5. Affine layer with 1024 output units

6. ReLU Activation Layer

7. Affine layer from 1024 input units to 10 outputs

Hint: some of this documentation will help!

Layers, Activations, Loss functions : https://www.tensorflow.org/api_guides/python/nn

BatchNorm: https://www.tensorflow.org/api_docs/python/tf/layers/batch_normalization


In [ ]:
# clear old variables
tf.reset_default_graph()

# define our input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.int64, [None])

def complex_model(X, y):
    Wconv = tf.get_variable("Wconv", shape=[7, 7, 3, 32])
    bconv = tf.get_variable("bconv", shape=[32])
    Waffine1 = tf.get_variable("Waffine1", shape=[5408, 1024])
    baffine1 = tf.get_variable("baffine1", shape=[1024])
    Waffine2 = tf.get_variable("Waffine2", shape=[1024, 10])
    baffine2 = tf.get_variable("baffine2", shape=[10])
    
    a1 = tf.nn.conv2d(X, Wconv, strides=[1, 1, 1, 1], padding='VALID') + bconv
    h1 = tf.nn.relu(a1)
    batchnorm = tf.nn.batch_normalization(h1)
    pool1act = tf.nn.max_pool(batchnorm, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    flatten1 = tf.reshape(pool1act, [-1, 5408])
    affine1act = tf.matmul(flatten1, Waffine1) + baffine1
    relu2act = tf.nn.relu(affine1act)
    y_out = tf.matmul(relu2act, Waffine2) + baffine2
    return y_out


## 3. Run your model!

Run your model using a `tf.Session()` and `feed_dict` on some data! You can choose whichever dataset you want, but if you want an easily-usable dataset, check out this MNIST tutorial which shows how to load in the data pretty easily:

https://www.oreilly.com/learning/not-another-mnist-tutorial-with-tensorflow

Have fun! Don't forget to create a training set and a testing set to evaluate your model!

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.model_selection import train_test_split
from tf.losses import sigmoid_cross_entropy
from tf.metrics import accuracy

In [ ]:
## Did not implement batching

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
X, y = mnist.train.images, mnist.train.labels
X_train, X_test, y_train, y_test = train_test_split(X, y)

result_train = complex_model(X_train, y_train)
result_test = complex_model(X_test, y_test)
accuracy_val = accuracy(result_test, y_test)

loss = tf.sigmoid_cross_entropy(result_train, y_train)
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(training_op, feed_dict={'X': X_train, 'y': y_train})
    acc = sess.run(accuracy_val, feed_dict={'X': X_test, 'y': y_test})
    print(acc)